# Darstellung des Temperaturverlaufes von 24h an einer Wetterstation

Von Lea Pellikan   |   Matrikel-Nr.: 3062311   |   Abgabe: 15.11.2020

Im folgendem Jupyter Notebook soll ein Barchart erstellt werden, das den stündlichen Temperaturverlauf eines ausgewählten Tages an einer ausgewählten Wetterstation darstellt. Die dafür verwendeten Daten stammen von der Quelle *************** . Diese Quelle wird von ******+ gepflegt und stellt, unter anderem, die stündlich gemessenen Temperaturen von allen Wetterstationen in Deutschland da.  Die Daten wurden aus Performanzgründen nicht direkt von Kafka importiert, sondern per CSV-Datei in eine MongoDB gespeichert. Aus dieser werden die benötigten Daten ausgelesen.

## Import der Libraries

Im folgendem wird die Library **pymongo** für die Verbindung zur MongoDB verwendet und **pandas** für das Verarbeiten der Collection. Außerdem wird **plotly.express** zur Darstellung der Daten verwendet. 

In [ ]:
import pandas as pd
import plotly.express as px
from pymongo import MongoClient

## Verbindungsaufbau zur MongoDB

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations
weather = client.bda.weather

In [ ]:
def get_collection(collection):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document)
    return documents

In [ ]:
stations_df = pd.DataFrame(get_collection(stations))
stations_df = stations_df.rename(columns={"STATION_ID": "station_id"})

In [ ]:
weather_df = pd.DataFrame(get_collection(weather))

In [ ]:
weather_stations = pd.merge(weather_df,stations_df, on="station_id", how="left")
weather_stations

In [ ]:
def temp_by_date(df,date,stationid):
    result= df.loc[df['date'].dt.normalize() == date ]
    result2 =  result.loc[result['station_id']== stationid ]
    return result2

In [ ]:
tempbydate = temp_by_date(weather_stations,"2020-08-20 00:00:00",917)
tempbydate

In [ ]:
def calculate_plot(tempbydate):
    fig2 =  px.bar(tempbydate,
             y= "temperature", #stunden
             x= "date",#temperatur
               title = 'Temperaturverlauf am ' + str(tempbydate["date"].dt.date.iloc[0]) + ' in ' + str(tempbydate["STATION_NAME"].iloc[0] ) ,
                hover_data=['station_id', 'STATION_NAME'], color='temperature')
    fig2.show()

In [ ]:
calculate_plot(tempbydate)